# Proyek Predictive Analytics: Diabetes-prediction
- Nama: Rezaldi
- Email: rezaldi30082003@students.amikom.ac.id
- Id Dicoding: rezaldi_20113717
- Dataset: https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset

## Import library

In [3]:
import pandas as pd

## Data Understanding

Tahap Data Understanding dalam proyek Machine Learning melibatkan beberapa langkah penting:

1. **Pengumpulan Data**: Ini melibatkan pengumpulan dataset yang akan digunakan dalam proyek. Dalam kasus ini, dataset telah disediakan dari Kaggle.

2. **Deskripsi Data**: Ini melibatkan pemahaman tentang data yang telah dikumpulkan. Misalnya, memahami jumlah fitur dalam dataset, jenis data (numerik atau kategorikal), dan jumlah sampel data.

3. **Eksplorasi Data**: Ini melibatkan visualisasi data untuk memahami distribusi dan hubungan antar fitur. Misalnya, menggunakan plot histogram, boxplot, atau scatterplot.

4. **Verifikasi Kualitas Data**: Ini melibatkan pemeriksaan kualitas data seperti keberadaan nilai yang hilang atau outlier, dan kebutuhan untuk pra-pemrosesan data.

5. **Pertanyaan Awal**: Mengidentifikasi pertanyaan atau hipotesis awal berdasarkan pemahaman awal tentang data.

Ingatlah bahwa pemahaman yang baik tentang data adalah kunci untuk membangun model Machine Learning yang efektif. Apakah ada hal lain yang Anda ingin tahu? 😊

### Data Collection

In [5]:
# Load the dataset into a DataFrame
data = pd.read_csv('diabetes_prediction_dataset.csv')
# Display the first 5 rows of the data
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


### Data Description

In [10]:
# Display the shape of the data
print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.')

The dataset contains 100000 rows and 9 columns.


In [11]:
# Display the data types of each column
print('\nThe data types of each column are:')
print(data.dtypes)


The data types of each column are:
gender                  object
age                    float64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object


In [13]:
# Display summary statistics for the numerical columns
data.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [14]:
# Display the number of missing values in each column
print('Number of missing values in each column:')
print(data.isnull().sum())

Number of missing values in each column:
gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64


In [15]:
# Display the number of unique values in each column
print('\nNumber of unique values in each column:')
print(data.nunique())


Number of unique values in each column:
gender                    3
age                     102
hypertension              2
heart_disease             2
smoking_history           6
bmi                    4247
HbA1c_level              18
blood_glucose_level      18
diabetes                  2
dtype: int64


In [7]:
# Display the frequency counts for categorical columns
for col in data.select_dtypes(include='object').columns:
    print(f'\nFrequency counts for {col}:')
    print(data[col].value_counts())

Number of missing values in each column:
gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

Number of unique values in each column:
gender                    3
age                     102
hypertension              2
heart_disease             2
smoking_history           6
bmi                    4247
HbA1c_level              18
blood_glucose_level      18
diabetes                  2
dtype: int64

Frequency counts for gender:
gender
Female    58552
Male      41430
Other        18
Name: count, dtype: int64

Frequency counts for smoking_history:
smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64
